In [1]:
import json
import requests
import csv
import configparser
import warnings
import uuid
import pandas as pd
import sqlite3 as sl
import re





c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sfera.inno.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [108]:
my_data  = pd.read_csv('my_data.secret', sep=',',index_col='parameters')
my_data.astype(str)

devUser=my_data.loc['devUser'].values [0]
devPassword=my_data.loc['devPassword'].values [0]
sferaUrlLogin=my_data.loc['sferaUrlLogin'].values [0]
sferaUrl=my_data.loc['sferaUrl'].values [0]
sferaTestCaseUrl=my_data.loc['sferaTestCaseUrl'].values [0]
sferaTSectionsUrl=my_data.loc['sferaTSectionsUrl'].values [0]
sferaSprintUrl=my_data.loc['sferaSprintUrl'].values [0]
sferaUrlSearch=my_data.loc['sferaUrlSearch'].values [0]





In [107]:
print(devUser)

vtb4068421@corp.dev.vtb


In [109]:
session = requests.Session()
session.post(sferaUrlLogin, json={"username": devUser, "password": devPassword}, verify=False)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sfera.inno.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [38]:
def getSuperSprintTasks(lst):
    # формируем список спринтов для запроса
    query_sprints = ''
    for sprint in lst:
        query_add = '%2C%27'
        query_end = '%27'
        if query_sprints == '':
            query_add = '%28%27'
        query_sprints += query_add + str(sprint) + query_end
    query_sprints += '%29'
    query = 'area%3D%27SKOKR%27%20and%20statusCategory%21%3D%27Done%27%20and%20type%20in%20%28%27task%27%29%20and%20sprint%20in%20' + query_sprints + '&size=1000&page=0&attributesToReturn=checkbox%2Cnumber%2Cname%2CactualSprint%2Cpriority%2Cstatus%2Cestimation%2Cspent%2Cassignee%2Cowner%2CdueDate%2CupdateDate%2CcreateDate%2CworkGroup'
    urlQuery = sferaUrlSearch + "?query=" + query
    #urlQuery = 'https://sfera.inno.local/app/tasks/api/v0.1/entities?query=area%3D%27SKOKR%27%20and%20statusCategory%21%3D%27Done%27%20and%20type%20in%20%28%27task%27%29%20and%20sprint%20in%20%28%274246%27%2C%274247%27%2C%274248%27%2C%274249%27%2C%274250%27%2C%274251%27%29&size=1000&page=0'
    response = session.get(urlQuery, verify=False)
    if response.ok != True:
        raise Exception("Error get sprint data " + response)

    tasks = json.loads(response.text)
    return tasks

def checkTasksEstimation(tasks):
    number = []
    name = []
    estimation = []
    sprint = []
    epic = []
    type = []
    for task in tasks['content']:
        number.append(task['number'])
        name.append(task['name'])
        estimation.append(int(task['estimation']/28800))
        sprint.append(task['actualSprint']['name'])
        type.append(task['workGroup'])
        if 'parentNumber' in task:
            epic.append(task['parentNumber'])
    tasks_df = pd.DataFrame({
        'number': number,
        'name': name,
        'estimation': estimation,
        'sprint': sprint,
        'epic': epic,
        'type': type
    })
    return tasks_df


tasks = getSuperSprintTasks(['4246', '4247', '4248', '4249', '4250', '4251'])
tasks_df = checkTasksEstimation(tasks)
print(tasks_df)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sfera.inno.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


         number                                               name  \
0    SKOKR-5977  Кастомизация комментариев по причинам отказов ...   
1    SKOKR-5976  Кастомизация комментариев по причинам отказов ...   
2    SKOKR-5975  Кастомизация комментариев по причинам отказов ...   
3    SKOKR-5974  Кастомизация комментариев по причинам отказов ...   
4    SKOKR-5973  Кастомизация комментариев по причинам отказов ...   
..          ...                                                ...   
214  SKOKR-5763  Бизнес мониторинг. СКМБ. Разработка - (Аналитика)   
215  SKOKR-5762  Внедрены в промышленную эксплуатацию новые ант...   
216  SKOKR-5761  Внедрены в промышленную эксплуатацию новые ант...   
217  SKOKR-5760              Внедрение ПВР-модели - (Планирование)   
218  SKOKR-5759  Бизнес мониторинг. СКМБ. Разработка - (Планиро...   

     estimation                             sprint       epic  \
0             1  Спринт 2024.2.6.NPP Shared Sprint  SCOR-2927   
1             1  Спринт 2024.

In [39]:
tasks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   number      219 non-null    object
 1   name        219 non-null    object
 2   estimation  219 non-null    int64 
 3   sprint      219 non-null    object
 4   epic        219 non-null    object
 5   type        219 non-null    object
dtypes: int64(1), object(5)
memory usage: 10.4+ KB


In [40]:
tasks_df['type'] = tasks_df['type'].astype(str)

In [41]:
tasks_df['name'] = tasks_df['name'].astype(str)

In [29]:
tasks_df['type'].tail

<bound method NDFrame.tail of 0      ['Новая функциональность']
1      ['Новая функциональность']
2      ['Новая функциональность']
3      ['Новая функциональность']
4      ['Новая функциональность']
                  ...            
214      ['Архитектурная задача']
215    ['Новая функциональность']
216    ['Новая функциональность']
217    ['Новая функциональность']
218      ['Архитектурная задача']
Name: type, Length: 219, dtype: object>

In [42]:
tasks_df.pivot_table(
    values='estimation',
    index='sprint',
    columns='type',
    aggfunc='sum',
    fill_value=0,
    margins=True, 
    margins_name='Total'
)

type,['Архитектурная задача'],['Линейная деятельность'],['Новая функциональность'],Total
sprint,,,,
Спринт 2024.2.1.NPP Shared Sprint,24,11,43,78
Спринт 2024.2.2.NPP Shared Sprint,24,10,44,78
Спринт 2024.2.3.NPP Shared Sprint,25,10,43,78
Спринт 2024.2.4.NPP Shared Sprint,25,8,45,78
Спринт 2024.2.5.NPP Shared Sprint,23,9,45,77
Спринт 2024.2.6.NPP Shared Sprint,19,7,50,76
Total,140,55,270,465


In [ ]:
tasks_df['type']